마스크 착용여부를 확인하는 모델을 만들어봅시다

https://www.kaggle.com/prithwirajmitra/covid-face-mask-detection-dataset

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
data_dir = '/content/drive/MyDrive/Tensorflow AI/New Masks Dataset'
img_height = 150
img_width = 150

Image Generator를 사용하여 학습데이터를 늘려봅시다

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
train_ds = train_datagen.flow_from_directory(
  data_dir+'/Train',    
  target_size=(img_height, img_width),
  class_mode = 'binary'
  )

Found 600 images belonging to 2 classes.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir+'/Validation',    
  image_size=(img_height, img_width))

Found 306 files belonging to 2 classes.


In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir+'/Test',    
  image_size=(img_height, img_width))

Found 100 files belonging to 2 classes.


pretrained model을 불러와봅시다

https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [ ]:
from tensorflow.keras.applications import VGG16
vgg_model = VGG16(weights = 'imagenet',include_top=False, input_shape=(150,150,3))

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
vgg_model.trainable = False

In [ ]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
from tensorflow.keras import Sequential, Model, layers
transfer_model = Sequential()

In [ ]:
transfer_model.add(vgg_model)

In [ ]:
transfer_model.add(layers.Flatten())
transfer_model.add(layers.Dense(256,activation='relu'))
transfer_model.add(layers.Dropout(0.5))
transfer_model.add(layers.Dense(1,activation='sigmoid'))

transfer_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
transfer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
history = transfer_model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
19/19 [==============================] - 344s 18s/step - loss: 0.6838 - acc: 0.6806 - val_loss: 11.0467 - val_acc: 0.8529
Epoch 2/10
19/19 [==============================] - 204s 11s/step - loss: 0.2781 - acc: 0.8869 - val_loss: 7.5811 - val_acc: 0.8889
Epoch 3/10
19/19 [==============================] - 204s 11s/step - loss: 0.1523 - acc: 0.9441 - val_loss: 4.1109 - val_acc: 0.9248
Epoch 4/10
19/19 [==============================] - 204s 11s/step - loss: 0.1675 - acc: 0.9435 - val_loss: 4.1123 - val_acc: 0.9379
Epoch 5/10
19/19 [==============================] - 204s 11s/step - loss: 0.1631 - acc: 0.9313 - val_loss: 7.8968 - val_acc: 0.9020
Epoch 6/10
19/19 [==============================] - 204s 11s/step - loss: 0.1350 - acc: 0.9546 - val_loss: 6.7183 - val_acc: 0.9248
Epoch 7/10
19/19 [==============================] - 204s 11s/step - loss: 0.1478 - acc: 0.9461 - val_loss: 7.7320 - val_acc: 0.9216
Epoch 8/10
19/19 [==============================] - 203s 11s/step - loss: 0

In [ ]:
vgg_model.trainable = True

# 기본 모델에 몇 개의 층이 있는지 확인 합니다.
print("Number of layers in the base model: ", len(vgg_model.layers))

Number of layers in the base model:  19


In [ ]:
# 해당 층 이후부터 미세 조정
fine_tune_at = 15

# `fine_tune_at` 층 이전의 모든 층을 고정
for layer in vgg_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
transfer_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['acc'])
transfer_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
history = transfer_model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
19/19 [==============================] - 229s 12s/step - loss: 0.2408 - acc: 0.9383 - val_loss: 10.1824 - val_acc: 0.9052
Epoch 2/10
19/19 [==============================] - 229s 12s/step - loss: 0.0942 - acc: 0.9682 - val_loss: 4.5731 - val_acc: 0.9739
Epoch 3/10
19/19 [==============================] - 230s 12s/step - loss: 0.0463 - acc: 0.9886 - val_loss: 5.0742 - val_acc: 0.9575
Epoch 4/10
19/19 [==============================] - 231s 12s/step - loss: 0.0215 - acc: 0.9917 - val_loss: 7.7675 - val_acc: 0.9542
Epoch 5/10
19/19 [==============================] - 230s 12s/step - loss: 0.0105 - acc: 0.9983 - val_loss: 3.8129 - val_acc: 0.9739
Epoch 6/10
19/19 [==============================] - 230s 12s/step - loss: 0.0037 - acc: 1.0000 - val_loss: 5.4323 - val_acc: 0.9706
Epoch 7/10
19/19 [==============================] - 230s 12s/step - loss: 0.0072 - acc: 0.9959 - val_loss: 7.7433 - val_acc: 0.9706
Epoch 8/10
19/19 [==============================] - 231s 12s/step - loss: 0